In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [2]:
df = pd.read_csv("./data/CaliforniaHousing/cal_housing.data", header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
X = df.iloc[:,:-1].values
y = np.log(df.iloc[:,-1].values)

### Train, test, scaler

In [7]:
from sklearn.preprocessing import MinMaxScaler # Rescales to have values between 0 and 1
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
scl = MinMaxScaler()
X_train_scl = scl.fit_transform(X_train)
X_test_scl = scl.transform(X_test)

### Custom loss function

**Goal:** Implement a custom loss function, for example, $L_1$ function.

In [36]:
def l1(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true-y_pred))

In [37]:
model = Sequential([
    Input(shape=X_train.shape[1:]), 
    Dense(10, activation='relu'), 
    Dense(1, activation='linear')
])

In [38]:
model.compile(loss=l1) # here we pass our custom loss function!

In [39]:
model.fit(X_train_scl, y_train, epochs=10)

Train on 15480 samples
Epoch 1/10
15480/15480 [==============================] - 1s 71us/sample - loss: 9.0745
Epoch 2/10
15480/15480 [==============================] - 1s 36us/sample - loss: 2.0949
Epoch 3/10
15480/15480 [==============================] - 1s 35us/sample - loss: 0.7246
Epoch 4/10
15480/15480 [==============================] - 1s 36us/sample - loss: 0.5696
Epoch 5/10
15480/15480 [==============================] - 1s 48us/sample - loss: 0.5070
Epoch 6/10
15480/15480 [==============================] - 1s 56us/sample - loss: 0.4666
Epoch 7/10
15480/15480 [==============================] - 1s 62us/sample - loss: 0.4306
Epoch 8/10
15480/15480 [==============================] - 1s 57us/sample - loss: 0.3963
Epoch 9/10
15480/15480 [==============================] - 1s 55us/sample - loss: 0.3633
Epoch 10/10
15480/15480 [==============================] - 1s 57us/sample - loss: 0.3334


In [42]:
model.predict(X_test_scl)

array([[11.43759 ],
       [11.773422],
       [12.58404 ],
       ...,
       [11.753563],
       [12.610883],
       [12.733097]], dtype=float32)

In [43]:
y_test

array([10.77268668, 10.73203937, 13.12236538, ..., 11.67589429,
       12.48028947, 13.08466151])

### Custom layer

**Goal:** Implement a custom layer with softplus activation function.

softplus($X$) = $\log(\exp(X)+1)$.

In [51]:
def softplus(x):
    return tf.math.log(tf.math.exp(x)+1)

In [52]:
softplus(t)

<tf.Tensor: id=41641, shape=(2,), dtype=float32, numpy=array([1.3132616, 2.126928 ], dtype=float32)>

In [66]:
# Necessary to add softplus function as a layer
from tensorflow.keras.layers import Lambda 
my_softplus_layer = Lambda(lambda x: softplus(x)) 

In [67]:
model = Sequential([
    Input(shape=X_train.shape[1:]), 
    Dense(10, activation='relu'), 
    Dense(1, activation=softplus), 
])

In [63]:
model.compile(loss=l1)

In [64]:
model.fit(X_train_scl, y_train, epochs=10)

Train on 15480 samples
Epoch 1/10
15480/15480 [==============================] - 1s 62us/sample - loss: 8.6969
Epoch 2/10
15480/15480 [==============================] - 1s 36us/sample - loss: 1.3418
Epoch 3/10
15480/15480 [==============================] - 1s 37us/sample - loss: 0.5844
Epoch 4/10
15480/15480 [==============================] - 1s 45us/sample - loss: 0.5296
Epoch 5/10
15480/15480 [==============================] - 1s 73us/sample - loss: 0.4957
Epoch 6/10
15480/15480 [==============================] - 1s 72us/sample - loss: 0.4669
Epoch 7/10
15480/15480 [==============================] - 1s 72us/sample - loss: 0.4392
Epoch 8/10
15480/15480 [==============================] - 1s 64us/sample - loss: 0.4116
Epoch 9/10
15480/15480 [==============================] - 1s 66us/sample - loss: 0.3869
Epoch 10/10
15480/15480 [==============================] - 1s 64us/sample - loss: 0.3635


### Small (but maybe useful) functions

In [68]:
z = tf.constant([3.,4.1,5.2])

In [70]:
tf.greater_equal(z, 3.5)

<tf.Tensor: id=74044, shape=(3,), dtype=bool, numpy=array([False,  True,  True])>

In [71]:
w = tf.constant([-1.1, 2, -4])

In [74]:
tf.where(tf.greater_equal(w,0), 2, 3) #if condition then value else other_value =ifelse()

<tf.Tensor: id=74058, shape=(3,), dtype=int32, numpy=array([3, 2, 3])>

In [78]:
def scaled_elu(x, slope=1.0, alpha=3.0):
    is_positive = tf.greater(x, 0.0)
    return slope*tf.where(is_positive, x, alpha*tf.exp(x)-alpha)

In [81]:
scaled_elu(tf.constant([-1., 3.]))

<tf.Tensor: id=74097, shape=(2,), dtype=float32, numpy=array([-1.8963616,  3.       ], dtype=float32)>

In [82]:
scaled_elu_tf = tf.function(scaled_elu)

In [83]:
scaled_elu_tf(tf.constant([-1., 3.]))

<tf.Tensor: id=74114, shape=(2,), dtype=float32, numpy=array([-1.8963616,  3.       ], dtype=float32)>

In [87]:
%timeit scaled_elu(tf.random.normal((1000,1000)))

30.5 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [88]:
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))

27.4 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
